# ほかの市場で値動きがあった後にS&P500先物がどう動くか

## 目的
S&P500先物の先行指標となりうる商品を探すこと

### 仮説
- 金利が上がったらS&P500も上がる。
- 原油や金が上がったらS&P500は下がる。
- ダウとS&P500が異なる動きをしたらその後修正の方向に価格が動く。

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import scipy
import scipy.stats as st
import statsmodels.stats.anova as anova
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import datetime as dt
from dateutil.relativedelta import relativedelta
import locale
from joblib import Parallel, delayed

import finalib as fl
import finalib.mine as mi
import ta

# 月や曜日を英語で取得するためこの設定をしておく
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

'en_US.UTF-8'

# データ読み込み

In [2]:
# 生データ
sp_dir = 'data/e-mini-sp500-200530'
df_sp_raw = pd.read_csv(f'{sp_dir}/e-mini-sp500-60min.csv')

dow_dir = 'data/e-mini-dow-200626'
df_dow_raw = pd.read_csv(f'{dow_dir}/e-mini-dow-60min.csv')

oil_dir = 'data/e-mini-crude-oil-200626'
df_oil_raw = pd.read_csv(f'{oil_dir}/e-mini-crude-oil-60min.csv')

gold_dir = 'data/gold-200626'
df_gold_raw = pd.read_csv(f'{gold_dir}/gold-60min.csv')

tnote2_dir = 'data/t-notes-2yr-200626'
df_tnote2_raw = pd.read_csv(f'{tnote2_dir}/t-notes-2yr-60min.csv')

tnote5_dir = 'data/t-notes-5yr-200626'
df_tnote5_raw = pd.read_csv(f'{tnote5_dir}/t-notes-5yr-60min.csv')

tnote10_dir = 'data/t-notes-10yr-200626'
df_tnote10_raw = pd.read_csv(f'{tnote10_dir}/t-notes-10yr-60min.csv')

tbond30_dir = 'data/t-bond-30yr-200629'
df_tbond30_raw = pd.read_csv(f'{tbond30_dir}/t-bond-30yr-60min.csv')

dfnames = ['sp', 'dow', 'oil', 'gold', 'tnote2', 'tnote5', 'tnote10', 'tbond30']
dfraws = [df_sp_raw, df_dow_raw, df_oil_raw, df_gold_raw, df_tnote2_raw, df_tnote5_raw, df_tnote10_raw, df_tbond30_raw]
#dfraws = [df_sp_raw, df_dow_raw, df_oil_raw, df_gold_raw]
#dfraws = [df_sp_raw, df_dow_raw]

In [3]:
dfs = []
for df in dfraws:
    dfs.append(df.copy())

In [4]:
def to_datetime_idx(df):
    df['DT'] = (df['Date'] + '-' + df['Time']).map(lambda s: dt.datetime.strptime(s, '%m/%d/%Y-%H:%M'))
    return df.set_index('DT')

for i, df in enumerate(dfs):
    dfs[i] = to_datetime_idx(df)

# ある商品が上がったらその後S&P500先物が上がるか

## 検証１
- （１）S&P500先物が上昇する確率
- （２）ある商品が上昇した次の足でS&P500先物が上昇する確率
- （２）が（１）より大きければ（統計的な判断は今はしない）その商品が正方向の先行指標として使える可能性がある。小さければ負方向の指標として使える可能性がある。

### 結果
- （２）が（１）より大きいと言える商品はなかった。一番大きかったのが`P(S&P UP | tnote2 UP) = 0.4890829694323144`だが、事前確率からの上昇は0.5%程度であり、ここにもし有意な差があったとしても、予測には使えない。
- 逆方向、つまり下落の予測についてもほぼ同じ結果になるはずのため試す必要はない。

## 検証２（仮）
T-Note10が上昇した足（足0）の次の3足以内でS&P500先物が足0の終値より高い終値をつける確率

## 検証1

In [5]:
# 値動きの列を加える
# 値動き　＝　終値　－　始値
# 上昇　＝　値動き　＞　０
# とする。

CODIFF_COL = 'CODiff'
ISUP_COL = 'isUp'

def add_CODiff(df):
    df[CODIFF_COL] = df['Close'] - df['Open']
    return df

def add_isUp(df):
    df[ISUP_COL] = df[CODIFF_COL] > 0
    return df

for i in range(len(dfs)):
    dfs[i] = add_CODiff(dfs[i])
    dfs[i] = add_isUp(dfs[i])

In [18]:
# 各商品の上昇列を取ってきて一つのDFにする
isUp_df = dfs[0].loc[:, [ISUP_COL]].rename(columns={ISUP_COL:dfnames[0]})
for i in range(1, len(dfs)):
    isUp_df = isUp_df.merge(dfs[i].loc[:, [ISUP_COL]].rename(columns={ISUP_COL:dfnames[i]}), how='outer', left_index=True, right_index=True)

isUp_df.shape

(127544, 8)

In [19]:
# NaNを含む行を全部削除
isUp_df = isUp_df[isUp_df.notnull().all(axis=1)]
isUp_df.shape

(87271, 8)

In [20]:
# ほかの列で上昇したときに、次の足で'sp'が上昇するかどうかを調べたい
# だから、'sp'の列だけ１つ繰り上げる（これで'sp'列については日時インデックスがずれることになることに注意）
isUp_df['sp'] = isUp_df['sp'].iloc[1:].append(pd.Series([np.nan])).to_numpy()
isUp_len = isUp_df.shape[0]
isUp_df.shape

(87271, 8)

In [30]:
# S&P500先物の上昇確率
print(f"P(S&P UP) = {isUp_df['sp'].sum() / isUp_len}")

P(S&P UP) = 0.4832762315087486


In [33]:
# ほかの商品が上昇した次の足でS&Pが上昇する確率
for col_name in isUp_df.columns.tolist()[1:]:
    ups_tmp = isUp_df[isUp_df[col_name]]
    print(f"P({col_name} UP) = {ups_tmp.shape[0] / isUp_len}")
    print(f"P(S&P UP | {col_name} UP) = {ups_tmp['sp'].sum() / ups_tmp.shape[0]}")

P(dow UP) = 0.4998911436788853
P(S&P UP | dow UP) = 0.4721037913171045
P(oil UP) = 0.4640143919515074
P(S&P UP | oil UP) = 0.4863810346956414
P(gold UP) = 0.4866106725028933
P(S&P UP | gold UP) = 0.4852238208491299
P(tnote2 UP) = 0.3384973244262126
P(S&P UP | tnote2 UP) = 0.4890829694323144
P(tnote5 UP) = 0.4356429971009843
P(S&P UP | tnote5 UP) = 0.4887030169125963
P(tnote10 UP) = 0.4384732614499662
P(S&P UP | tnote10 UP) = 0.4889196675900277
P(tbond30 UP) = 0.43785449920363007
P(S&P UP | tbond30 UP) = 0.48916570710771484
